In [1]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import *
from accuracy import *
from transformers import get_linear_schedule_with_warmup
from model import Model

import numpy as np
import torch
import matplotlib.pyplot as plt

def train(model, num_actions, batch_size, epochs=25, lr=0.001, MAX_DETECTIONS=20):
    dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    train_size = int(len(dataset) * (2/3))
    valid_size = int(len(dataset) - train_size)
    
    print("Training Dataset Size: {}, Validation Dataset Size: {}".format(train_size, valid_size))
    
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])
    
    collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(0.2*epochs), epochs)

    train_loss = np.zeros(epochs)
    valid_loss = np.zeros(epochs)
    
    train_accuracy = np.zeros(epochs)
    valid_accuracy = np.zeros(epochs)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_loss = 0.
        num_batches = 0
        
        total = 0
        correct = 0
                
        for data in train_dataloader:
            _, bboxes, features, steps, entities, entity_count, _, _ = data
            
            # Zero out any gradients.
            optimizer.zero_grad()
            
            # Run inference (forward pass).
            loss_data, VG, RR = model(batch_size, num_actions, steps, features, bboxes, entities, entity_count)
            
            # Loss from alignment.
            loss_ = compute_loss_batched(loss_data)

            # Backpropagation (backward pass).
            loss_.backward()

            # Update parameters.
            optimizer.step()
            
            epoch_loss += loss_
            num_batches += 1
            
        # Scheduler update.
        scheduler.step()
        epoch_loss = epoch_loss / (num_batches * batch_size)
        
        # Save loss and accuracy at each epoch and plot.
        train_loss[epoch] = float(epoch_loss)
        train_accuracy[epoch] = get_alignment_accuracy(model, train_dataloader, batch_size, num_actions) 
        
        valid_loss[epoch] = get_alignment_loss(model, valid_dataloader, batch_size, num_actions)
        valid_accuracy[epoch] = get_alignment_accuracy(model, valid_dataloader, batch_size, num_actions)

        print("Epoch {} - Train Loss: {:.2f}, Validation Loss: {:.2f}, Train Accuracy: {:.2f}, Validation Accuracy: {:.2f}"
              .format(epoch + 1, train_loss[epoch], valid_loss[epoch], train_accuracy[epoch], valid_accuracy[epoch]))
    
    plt.figure()
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.legend()
    
    plt.figure()
    plt.plot(train_accuracy, label='train accuracy')
    plt.plot(valid_accuracy, label='valid accuracy')
    plt.legend()
    
    plt.show()
        
    return train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(device)

In [ ]:
# Trainer.

train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data = train(model, 8, 4, epochs=500, lr=1e-6)

Training Dataset Size: 38, Validation Dataset Size: 19
Epoch 1 - Train Loss: 1225.46, Validation Loss: 1132.88, Train Accuracy: 0.51, Validation Accuracy: 0.50
Epoch 2 - Train Loss: 1219.69, Validation Loss: 1162.51, Train Accuracy: 0.50, Validation Accuracy: 0.49
Epoch 3 - Train Loss: 1216.06, Validation Loss: 1158.76, Train Accuracy: 0.48, Validation Accuracy: 0.50
Epoch 4 - Train Loss: 1230.95, Validation Loss: 1144.92, Train Accuracy: 0.50, Validation Accuracy: 0.51
Epoch 5 - Train Loss: 1200.11, Validation Loss: 1160.29, Train Accuracy: 0.49, Validation Accuracy: 0.49
Epoch 6 - Train Loss: 1215.41, Validation Loss: 1188.60, Train Accuracy: 0.49, Validation Accuracy: 0.52
Epoch 7 - Train Loss: 1222.61, Validation Loss: 1161.85, Train Accuracy: 0.49, Validation Accuracy: 0.50
Epoch 8 - Train Loss: 1218.94, Validation Loss: 1151.75, Train Accuracy: 0.50, Validation Accuracy: 0.48
Epoch 9 - Train Loss: 1219.54, Validation Loss: 1173.54, Train Accuracy: 0.49, Validation Accuracy: 0.52


In [ ]:
# Evaluation.

from eval_fi import eval_all_dataset
eval_all_dataset(model, path="/h/sagar/ece496-capstone/datasets/fi_reduced")

In [ ]:
# Visualizer.

from visualizer import inference

YCII = "/h/sagar/ece496-capstone/datasets/ycii"
FI = "/h/sagar/ece496-capstone/datasets/fi"

VG, RR = inference(model, 27, 0, FI)

In [ ]:
# Saving and loading weights.

SAVE = False
LOAD = True

if SAVE:
    torch.save(model.state_dict(), "/h/sagar/ece496-capstone/weights/t1")
    
if LOAD:
    model.load_state_dict(torch.load("/h/sagar/ece496-capstone/weights/t1"))

In [ ]:
# Reload modules.

import importlib
import visualizer
import eval_fi
import model as mdl
import loss

importlib.reload(visualizer)
importlib.reload(eval_fi)
importlib.reload(mdl)
importlib.reload(loss)